# DM lesson4 :

Dans ce dataset: https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv, chaque ligne correspond à un produit alimentaire mis en vente par un utilisateur.

Objectif: cleaner le dataset.

On aimerait avoir une colonne de prix unifiés en euros. Problème: la currency n'est pas indiquée pour tous les produits: il va falloir essayer de "deviner" les currency manquantes, en se basant sur l'adresse IP de l'utilisateur.
La colonne "infos" liste des ingrédients présents dans le produit. On préfèrerait avoir une colonne de type bool par ingrédient, indiquant si le produit contient ou non cet ingrédient.
Voic une liste d'APIs qui peut vous être utile : https://github.com/public-apis/public-apis (mais vous pouvez en utiliser d'autres si vous le voulez).

In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests as req

In [3]:
import json

In [46]:
df_products = pd.read_csv('https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv', sep=';',  error_bad_lines=False)

In [47]:
#Create an empty contry code column
df_products['currencyCode'] = np.nan
df_products['price_EUR'] = np.nan

In [48]:
df_products.head()

,username,ip_address,product,price,infos,currencyCode,price_EUR
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,NaN,NaN
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,NaN,NaN
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,NaN,NaN
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,NaN,NaN
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",NaN,NaN


In [49]:
eur_doll_change = 0.842
localhost_ip = '137.194.95.136'

def get_ip_currency(df_row):
    
    #print(df_row)
    url = 'http://www.geoplugin.net/json.gp?ip=' + df_row.ip_address 
    ip_content = req.get(url).content
    
    if(len(ip_content) != 0 and len(df_row.price.split(' ')) == 1): #<pour s'assurer que le résultat de la requete est correct
        
        data = json.loads(ip_content)
        
       #Pour s'assurer que l'ip demandée est différente de l'ip locale (éviter erreurs 'bad ip_address')
        if(data['geoplugin_request'] != localhost_ip ):
            
            #print(data['geoplugin_request'])
            #print('\t' + str(data['geoplugin_currencyConverter']))
            df_row.currencyCode = data['geoplugin_currencyCode']
            
            return data['geoplugin_currencyCode']
    
    return None 


In [ ]:
df_products['currencyCode'] = df_products.apply(get_ip_currency, axis=1)

In [13]:
def get_currency_code(df_row):
    
    price = df_row.price.split(' ')
    
    if(len(price) > 1):
        curr_code = price[1]
    else:
        curr_code = df_row.currencyCode
    
    return curr_code

In [14]:
df_products['currencyCode'] = df_products.apply(get_currency_code, axis=1)

In [15]:
def convert_to_euro(df_row, eur_exch_rate):
    
    if(df_row.currencyCode in eur_exch_rate['rates'].keys()):
        
        price = df_row.price.split(' ')[0]
        eur_price = float(price) / eur_exch_rate['rates'][df_row.currencyCode]
        
        return eur_price
    else:
        
        return np.nan

In [16]:
api_url = req.get('https://api.exchangerate-api.com/v4/latest/EUR').content
eur_exch_rate = json.loads(api_url)

In [17]:
df_products['price_EUR'] = df_products.apply(convert_to_euro, eur_exch_rate=eur_exch_rate,axis=1)

In [19]:
df_products.head(20)

,username,ip_address,product,price,infos,currencyCode,price_EUR
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,None,NaN
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,None,NaN
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,None,NaN
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,USD,314.377601
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JPY,7.854686
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33 MGA,Ingredients: sugar and milk,MGA,NaN
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar,None,NaN
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MXN,5.238970
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MXN,4.134916
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish",USD,773.262057


----------------------------------

### 2 - On préfèrerait avoir une colonne de type bool par ingrédient, indiquant si le produit contient ou non cet ingrédient.

In [20]:
df_products['ingredients'] = df_products.infos.str.split(' ')

In [21]:
df_products.head()

,username,ip_address,product,price,infos,currencyCode,price_EUR,ingredients
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,None,NaN,"[May, contain, sugar]"
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,None,NaN,"[Contains, peanut, and, fish]"
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,None,NaN,"[Ingredients:, mustard, and, fish]"
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,USD,314.377601,"[Contains, gluten]"
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JPY,7.854686,"[May, contain, sugar,, egg, and, fish]"


In [22]:

def get_indgredients_names(df_row, ingredients_names):
    
    for ingredient in df_row.ingredients:
        
        if ingredient.lower() not in ingredients_names :
            ingredients_names.append(ingredient.lower())
    
    

In [23]:
list_ingredients_names = []
a = df_products.apply(get_indgredients_names, ingredients_names=list_ingredients_names, axis=1)

In [24]:
print(list_ingredients_names)

['may', 'contain', 'sugar', 'contains', 'peanut', 'and', 'fish', 'ingredients:', 'mustard', 'gluten', 'sugar,', 'egg', 'milk', 'peanut,', 'mustard,', 'gluten,', 'soja,', 'soja']


In [25]:
to_remove = ['may', 'contain', 'contains', 'and', 'ingredients:']

In [26]:
for word in to_remove:
    if word in list_ingredients_names:
        list_ingredients_names.remove(word)

In [27]:
print(list_ingredients_names)

['sugar', 'peanut', 'fish', 'mustard', 'gluten', 'sugar,', 'egg', 'milk', 'peanut,', 'mustard,', 'gluten,', 'soja,', 'soja']


### On crée les nouvelles colonnes et on les remplit

In [ ]:
str

In [42]:
def fill_ingredient_column(df_row, column_name):
    
    ingredients_list = [word.lower() for word in df_row['ingredients']]
    
    if column_name in ingredients_list:
        return True
    else:
        return False

In [43]:
for word in list_ingredients_names:
    df_products[word] = df_products.apply(fill_ingredient_column, column_name=word, axis=1)

In [45]:
df_products.head()

,username,ip_address,product,price,infos,currencyCode,price_EUR,ingredients,sugar,peanut,...,mustard,gluten,"sugar,",egg,milk,"peanut,","mustard,","gluten,","soja,",soja
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,None,NaN,"[May, contain, sugar]",True,False,...,False,False,False,False,False,False,False,False,False,False
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,None,NaN,"[Contains, peanut, and, fish]",False,True,...,False,False,False,False,False,False,False,False,False,False
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,None,NaN,"[Ingredients:, mustard, and, fish]",False,False,...,True,False,False,False,False,False,False,False,False,False
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,USD,314.377601,"[Contains, gluten]",False,False,...,False,True,False,False,False,False,False,False,False,False
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JPY,7.854686,"[May, contain, sugar,, egg, and, fish]",False,False,...,False,False,True,True,False,False,False,False,False,False
